**Analysis owner**: Tamiris Crepalde <br>
**Date**: 09/01/2023 <br>

This analysis pretends to do an Exploratory Data Analysis of a dateset related to cancellations of travel packages.

Packages are a type of travel product that include flight tickets and hotel reservations.

### Setup

In [1]:
import os

import numpy as np
import pandas as pd

import src.utils as utils
from GoogleUtils import GoogleUtils

%load_ext autoreload
%autoreload 2

In [2]:
gutils = GoogleUtils()

### Load data

In [5]:
query = utils.read_file('src/queries/package_cancellation_data.sql')
cancel_date = gutils.read_from_bq(query)

In [6]:
cancel_date.head()

,operation_order_id,operation_id,order_id,order_date,origin_city,order_origin_city,origin_state,origin_country,destination_type,destination_city,...,first_valid_date_brt,last_valid_date_brt,first_possible_wish_date_brt,first_wish_date_brt,second_wish_date_brt,third_wish_date_brt,operation_status_id,operation_status_name,last_update_status_date_brt,qty_status_changes
0,8731462311137516076,-1.029609e+18,8638330343881013924,2019-01-10,Vitória,Vitória,Espírito Santo,Brasil,Nacional,Maceió,...,2019-04-01,2019-11-30,None,2019-11-30,2019-11-30,2019-11-30,2.0,Cotando aéreo,2019-07-15,2
1,1666196932304516294,-6.998555e+18,-387854366642617686,2019-10-26,Natal,Natal,Rio Grande do Norte,Brasil,Nacional,Curitiba,...,2020-03-01,2020-11-30,None,2020-04-01,2020-04-06,2020-04-25,14.0,Cancelada,2020-09-12,3
2,-3544610697494632345,NaN,-93571207230795879,2019-11-12,Salvador,Salvador,Bahia,Brasil,Nacional,Rio de Janeiro,...,2020-03-01,2020-11-30,None,2020-08-31,2020-10-12,2020-11-23,NaN,None,2021-05-19,4
3,963178260747839671,NaN,5566700318165541820,2021-01-10,Salvador,Salvador,Bahia,Brasil,Nacional,Gramado,...,2021-09-01,2021-10-20,None,2021-09-04,2021-09-28,2021-10-09,NaN,None,2021-03-25,3
4,3857852037455538925,NaN,1477871034842568547,2021-09-27,Recife,Recife,Pernambuco,Brasil,Nacional,Bonito,...,2022-03-01,2022-11-30,None,2022-09-09,2022-10-20,2022-11-01,NaN,None,2022-08-05,2


We have a lot of null values:

In [20]:
cancel_date.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 619174 entries, 0 to 619173
Data columns (total 28 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   operation_order_id            619174 non-null  int64  
 1   operation_id                  150755 non-null  float64
 2   order_id                      619174 non-null  int64  
 3   order_date                    619174 non-null  object 
 4   origin_city                   299421 non-null  object 
 5   order_origin_city             616351 non-null  object 
 6   origin_state                  299421 non-null  object 
 7   origin_country                619174 non-null  object 
 8   destination_type              619174 non-null  object 
 9   destination_city              619174 non-null  object 
 10  destination_state             619174 non-null  object 
 11  destination_country           619174 non-null  object 
 12  qty_people                    619174 non-nul

- The feature `operation_id` indicates if the order started the process of being operated before the cancellation and the features `operation_status_id` and `operation_status_name` indicates the status of the operation process. The first feature we'll convert to binary because we want to analyze if the client cancelled the order after the operation process started. 
- `origin_city` and `origin_state` miss more than half of the data, but `order_origin_city` has the same purpose of `origin_city`. So, we'll consider the `order_origin_city` valid and we'll use it to try to fill up the missing data of `origin_state`. 
- The feature `filled_form` is related to the client inform the desired dates to travel, so is enough if this information is binary too. `first_form_fill_date_brt` and `last_form_fill_date_brt` are related to the first and last date in which the client filled up the form, we'll use these features only to verify if the process of filling up the form and the cancellation are close events.
- `first_possible_wish_date_brt`is completely empty, so we'll remove the column.
- The columns `first_wish_date_brt`, `second_wish_date_brt` and `third_wish_date_brt` are the wished dates to travel informed by half of the clients, at a first moment we'll not be concern about these information.

Next we'll proceed to execute this modifications.

In [32]:
(cancel_date.isna().sum()/cancel_date.shape[0])*100

operation_order_id                0.000000
operation_id                     75.652240
order_id                          0.000000
order_date                        0.000000
origin_city                      51.641865
order_origin_city                 0.455930
origin_state                     51.641865
origin_country                    0.000000
destination_type                  0.000000
destination_city                  0.000000
destination_state                 0.000000
destination_country               0.000000
qty_people                        0.000000
qty_dailies                       0.001292
accommodation_type               10.992871
filled_form                      51.617962
first_form_fill_date_brt         51.617962
last_form_fill_date_brt          51.617962
first_valid_date_brt              0.000969
last_valid_date_brt               0.001131
first_possible_wish_date_brt    100.000000
first_wish_date_brt              51.619416
second_wish_date_brt             51.619416
third_wish_